# Final Project: Home Credit Default Risk Competition

## *In this final project, we will build the best model (as judged by the ROC_AUC score) using the Home Credit default dataset. The objective of this data science project is to use historical loan application data to predict whether or not an applicant will be able to repay a loan. This is a standard supervised classification task:*
- Supervised: The labels are included in the training data and the goal is to train a model to learn to predict the labels from th饿features
- Classification: The label is a binary variable, 0 (will repay loan on time), 1 (will have difficulty repaying loan)

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
# load dataset
df = pd.read_csv('./datasets/application_train.csv')

### EDA analysis

In [3]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# get describtion of columns
des = pd.read_csv('./datasets/HomeCredit_columns_description.csv', encoding = "ISO-8859-1", sep=',')

In [5]:
des.head()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [6]:
des.drop(['Unnamed: 0','Table'], axis=1, inplace=True)

In [9]:
#!pip install tabulate

#print(des.to_markdown())

In [27]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(df.dtypes)

In [28]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(df)
categorical_columns = categorical_columns_selector(df)
    

In [31]:
#!pip install pandas_profiling
from pandas_profiling import ProfileReport

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl (262 kB)
     |████████████████████████████████| 124 kB 723 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached multimethod-1.8-py3-none-any.whl (9.8 kB)
     |████████████████████████████████| 292 kB 713 kB/s eta 0:00:01
  Using cached visions-0.7.4-py3-none-any.whl (102 kB)
  Using cached missingno-0.5.1-py3-none-any.whl (8.7 kB)
     |████████████████████████████████| 2.4 MB 1.7 MB/s eta 0:00:01
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
     |████████████████████████████████| 673 kB 721 kB/s eta 0:00:01
  Using cached tangled_up_in_unicode-0.2.0-py3-none-any.whl (4.7 MB)
     |████████████████████████████████| 2.0 MB 1.9 MB/s eta 0:00:01
  Using cached ImageHash-4.2.1.tar.gz (812 kB)
     |████████████████████████████████| 4.3 MB 1.6 MB/s eta 0:00:01
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.wh

In [32]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [ ]:
profile.to_file("final.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/zheng/tensorflow-test/env/lib/python3.8/site-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
/Users/zheng/tensorflow-test/env/lib/python3.8/site-packages/scipy/stats/stats.py:4814: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(var) / np.sqrt(2)))
/Users/zheng/tensorflow-test/env/lib/python3.8/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/Users/zheng/tensorflow-test/env/lib/python3.8/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Count": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(includ

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]